# CaseLaw dataset to assist with Law-Research - EDA
---
<dl>
  <dt>Acquiring the dataset</dt>
  <dd>We initially use dataset of all cases in USA to be able to train it and as a proof of concept.</dd>
  <dd>The dataset is available in XML format, which we will put in mongodb or firebase format based on how unstructured the dataset is.</dd>
    <dd>dataset url: (https://case.law/)
</dd>

  <dt>Research</dt>
  <dd>We are looking into <em>NLP</em>, <em>LSTM</em> and <em>Sentiment Analysis</em>.</dd>
</dl>

In [1]:
import jsonlines
from pymongo import MongoClient

In [2]:
# client = MongoClient()
client = MongoClient()
db = client.legal_ai
cases = db.cases

In [3]:
some_date = '1820-01'

In [4]:
print(int(some_date[0:4])<1950)

True


In [5]:
id_saved = []
with jsonlines.open('data.jsonl') as reader:
    for obj in reader:
        if int(obj['decision_date'][0:4])>1950:
            case_id = cases.insert_one(obj).inserted_id
            id_saved.append(case_id)

In [6]:
len(id_saved)

30075

In [7]:
# 2 Keyword generation?
import enchant
d = enchant.Dict("en_US")
d.synonym("contract")

AttributeError: 'Dict' object has no attribute 'synonym'

In [8]:
from py_thesaurus import Thesaurus

input_word = "dream"

new_instance = Thesaurus(input_word)

# Get the synonyms according to part of speech
# Default part of speech is noun

print(new_instance.get_synonym())

print(new_instance.get_synonym(pos='verb'))

print(new_instance.get_synonym(pos='adj'))

No Internet Connection
[]
No Internet Connection
[]
No Internet Connection
[]


In [60]:
from PyDictionary import PyDictionary
dictionary=PyDictionary()
print (dictionary.meaning("indentation"))

[]


## Testing out Similarity Mechanism
---
### Setup
- Test PyDictionary to build keywords
- Construct a mechanism, to extract keywords, and store in a searchable manner.
---
### Search
- Build keywords out of your search
- Search among dataset keywords
- Nearest dates, highest weight, highest precidence shows up
- Pagination scroll, continues the search.

In [6]:
# Assume this dataset
dataset = ["The contract was breached by the defendant", "The defendant assaulted the victim on multiple occassions"]

In [30]:
# Build keywords for this dataset. 
from rake_nltk import Rake
from nltk.corpus import stopwords 
r = Rake() # Uses stopwords for english from NLTK, and all puntuation characters.
keyword_dataset = []

In [31]:
text="The contract was breached by the defendant"
for each_text in dataset:
    a=r.extract_keywords_from_text(each_text)
    b=r.get_ranked_phrases()
    c=r.get_ranked_phrases_with_scores()
    keyword_dataset.append(c)

In [32]:
keyword_dataset

[[(1.0, 'defendant'), (1.0, 'contract'), (1.0, 'breached')],
 [(4.0, 'multiple occassions'), (4.0, 'defendant assaulted'), (1.0, 'victim')]]

In [34]:
from PyDictionary import PyDictionary
dictionary=PyDictionary()


In [35]:
dictionary.meaning("defendant")

KeyboardInterrupt: 

In [36]:
dictionary.synonym("Life")

Life has no Synonyms in the API


/Users/flynsequeira/anaconda3/lib/python3.7/site-packages/PyDictionary/utils.py:5: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 5 of the file /Users/flynsequeira/anaconda3/lib/python3.7/site-packages/PyDictionary/utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  return BeautifulSoup(requests.get(url).text)


## Transforming dataset
---
### Extract the first data and study it
- Identify the key elements that need to be transformed & list them
- Build a mechanism to transform for one datapoint.
---
### Perform for entire dataset
- Run a loop and apply the same changes for every datapoints.

In [23]:
# Extracting the first element
first_case = cases.find_one()

{'_id': ObjectId('5c0588bd08ef95023a285d30'),
 'id': 1611959,
 'name': 'Arkansas Game and Fish Commission v. Edgmon',
 'name_abbreviation': 'Arkansas Game & Fish Commission v. Edgmon',
 'decision_date': '1951-01-15',
 'docket_number': '4-9304',
 'first_page': '207',
 'last_page': '211',
 'citations': [{'type': 'parallel', 'cite': '235 S.W.2d 554'},
  {'type': 'official', 'cite': '218 Ark. 207'}],
 'volume': {'volume_number': '218'},
 'reporter': {'full_name': 'Arkansas Reports'},
 'court': {'id': 8808,
  'name': 'Arkansas Supreme Court',
  'name_abbreviation': 'Ark.',
  'jurisdiction_url': None,
  'slug': 'ark'},
 'jurisdiction': {'id': 34,
  'slug': 'ark',
  'name': 'Ark.',
  'name_long': 'Arkansas',
  'whitelisted': True},
 'casebody': {'data': '<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<casebody xmlns="http://nrs.harvard.edu/urn-3:HLS.Libr.US_Case_Law.Schema.Case_Body:v1" xmlns:xlink="http://www.w3.org/1999/xlink" firstpage="207" lastpage="211"> <docketnumber id="b231-6" pgmap="23

In [24]:
first_case['casebody']['data']

'<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<casebody xmlns="http://nrs.harvard.edu/urn-3:HLS.Libr.US_Case_Law.Schema.Case_Body:v1" xmlns:xlink="http://www.w3.org/1999/xlink" firstpage="207" lastpage="211"> <docketnumber id="b231-6" pgmap="231">4-9304</docketnumber> <citation id="A2j" pgmap="231">235 S. W. 2d 554</citation> <parties id="b231-5" pgmap="231">Arkansas Game and Fish Commission v. Edgmon.</parties> <decisiondate id="b231-7" pgmap="231">Opinion delivered January 15, 1951.</decisiondate> <attorneys id="b232-4" pgmap="232">Ed E. Ashbaugh, for appellant.</attorneys> <attorneys id="b232-5" pgmap="232">Neill Bohlinger, for intervening appellant.</attorneys> <opinion type="majority"> <author id="b232-6" pgmap="232">Grieein Smith, Chief Justice.</author> <p id="ASCi" pgmap="232">The executive secretary of the State Game and Fish, Commission declined to authorize payment of $105 to four persons who claimed to have killed seven wolves over six months old, as evidenced by certificate

In [30]:
first_case['casebody']['data']

'<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<casebody xmlns="http://nrs.harvard.edu/urn-3:HLS.Libr.US_Case_Law.Schema.Case_Body:v1" xmlns:xlink="http://www.w3.org/1999/xlink" firstpage="207" lastpage="211"> <docketnumber id="b231-6" pgmap="231">4-9304</docketnumber> <citation id="A2j" pgmap="231">235 S. W. 2d 554</citation> <parties id="b231-5" pgmap="231">Arkansas Game and Fish Commission v. Edgmon.</parties> <decisiondate id="b231-7" pgmap="231">Opinion delivered January 15, 1951.</decisiondate> <attorneys id="b232-4" pgmap="232">Ed E. Ashbaugh, for appellant.</attorneys> <attorneys id="b232-5" pgmap="232">Neill Bohlinger, for intervening appellant.</attorneys> <opinion type="majority"> <author id="b232-6" pgmap="232">Grieein Smith, Chief Justice.</author> <p id="ASCi" pgmap="232">The executive secretary of the State Game and Fish, Commission declined to authorize payment of $105 to four persons who claimed to have killed seven wolves over six months old, as evidenced by certificate

In [28]:
import xml.etree.ElementTree as ET
root = ET.fromstring(first_case['casebody']['data'])

In [29]:
root

<Element '{http://nrs.harvard.edu/urn-3:HLS.Libr.US_Case_Law.Schema.Case_Body:v1}casebody' at 0x1137d4ef8>

In [31]:
for child in root:
    print(child.tag[child.tag.index("}")+1:],":",child.text)
    print("===========================================")
    for sub_child in child:
        print("     "+sub_child.tag[sub_child.tag.index("}")+1:], ":", sub_child.text)
        print('--------------------------')
        for sub_sub_child in sub_child:
            print("     "+sub_sub_child.tag[sub_sub_child.tag.index("}")+1:],":",sub_sub_child.text)

docketnumber : 4-9304
citation : 235 S. W. 2d 554
parties : Arkansas Game and Fish Commission v. Edgmon.
decisiondate : Opinion delivered January 15, 1951.
attorneys : Ed E. Ashbaugh, for appellant.
attorneys : Neill Bohlinger, for intervening appellant.
opinion :  
     author : Grieein Smith, Chief Justice.
--------------------------
     p : The executive secretary of the State Game and Fish, Commission declined to authorize payment of $105 to four persons who claimed to have killed seven wolves over six months old, as evidenced by certificate of the Boone County Court. See Act 183 of 1949. The statute is entitled “An Act to authorize . . . counties ... to pay bounties for the killing of wolves and to provide that the State . . . shall pay an equal sum as a bounty, and for other purposes.” The emergency clause is a finding that farmers are suffering irreparable damage ‘£ from wolves destroying cattle and other live stock.” The measure received two-thirds of the votes of all members 

In [ ]:
# dude a vs dude b
# petitioner vs respondent 